In [1]:
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import torch
from models.siamese import EmbeddingModel, embed_func
from sklearn.model_selection import train_test_split
import pandas as pd
from test_study.coherence.features import TextFeatureExtractor

/Users/seanlavi/dev/venvs/Schizophrenic_Speech/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertModel were not initialized from the model checkpoint at onlplab/alephbert-base and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertModel were not initialized from the model checkpoint at onlplab/alephbert-base and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and in

In [2]:
feat_extractor = TextFeatureExtractor()
data_df = pd.read_csv("/Users/seanlavi/dev/Schizophrenic_Speech/data/clean_data.csv", index_col = False)
data_df = feat_extractor.transform_data_to_train_schema(data_df).dropna()
train, test = train_test_split(data_df, test_size = 0.2, random_state = 42)
model = EmbeddingModel.load_model("/Users/seanlavi/dev/Schizophrenic_Speech/models/embedding_model.pth")

Model loaded and ready for inference or further training


/Users/seanlavi/dev/Schizophrenic_Speech/models/siamese.py:65: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(path)


In [3]:
train_embeddings = embed_func(np.array(train['answer'].values.tolist()))
train_tensor = torch.tensor(train_embeddings, dtype = torch.float32)
with torch.no_grad():  # Disable gradient calculation for inference
    X_train = model(train_tensor)

test_embeddings = embed_func(np.array(test['answer'].values.tolist()))
test_tensor = torch.tensor(test_embeddings, dtype = torch.float32)

with torch.no_grad():
    X_test = model(test_tensor)

# Convert to tensor
train_tensor = torch.tensor(train_embeddings, dtype = torch.float32)

# Predict
X_train = model.predict(train_tensor)

# Similarly for test data
test_tensor = torch.tensor(test_embeddings, dtype = torch.float32)
X_test = model.predict(test_tensor)

y_train = np.array(train['label'].values.tolist())
y_test = np.array(test['label'].values.tolist())

/var/folders/b1/frm8p4wx2v1_r2r5qhspvnm80000gn/T/ipykernel_14320/1845083558.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_tensor = torch.tensor(train_embeddings, dtype = torch.float32)
/var/folders/b1/frm8p4wx2v1_r2r5qhspvnm80000gn/T/ipykernel_14320/1845083558.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_tensor = torch.tensor(test_embeddings, dtype = torch.float32)
/var/folders/b1/frm8p4wx2v1_r2r5qhspvnm80000gn/T/ipykernel_14320/1845083558.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_tensor = torch.te

In [4]:
knn = KNeighborsClassifier(n_neighbors = 2, metric = 'euclidean')
svc = LinearSVC()

knn.fit(X_train, y_train)
svc.fit(X_train, y_train)

y_pred_knn = knn.predict(X_test)
acc_knn = accuracy_score(y_test, y_pred_knn)
y_pred_svc = svc.predict(X_test)
acc_svc = accuracy_score(y_test, y_pred_svc)

print(f'KNN accuracy = {acc_knn}, SVM accuracy = {acc_svc}')
print(metrics.classification_report(list(y_test), list(y_pred_knn)))


KNN accuracy = 0.8501742160278746, SVM accuracy = 0.8536585365853658
              precision    recall  f1-score   support

           0       0.84      0.98      0.90       208
           1       0.91      0.51      0.65        79

    accuracy                           0.85       287
   macro avg       0.87      0.74      0.78       287
weighted avg       0.86      0.85      0.83       287



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

X_embedded = TSNE(n_components=2).fit_transform(X_test)

plt.figure(figsize=(10,10))

for i, t in enumerate(set(y_test)):
    idx = y_test == t
    plt.scatter(X_embedded[idx, 0], X_embedded[idx, 1], label=t)   

plt.legend(bbox_to_anchor=(1, 1));